<a href="https://colab.research.google.com/github/TimoZhou1024/AI-Painting/blob/main/DreamBooth_Stable_Diffusion_(NovelAILeaks_Ver_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
#@title Install Requirements

# !curl -LO https://github.com/huggingface/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
# use memory-optim version
!curl -Lo train_dreambooth.py https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/dreambooth.py
!pip install -U pip

!pip install -qq git+https://github.com/ShivamShrirao/diffusers
!pip install -qq -U --pre triton
!pip install -qq accelerate==0.12.0 transformers ftfy bitsandbytes gradio
!pip install -qq omegaconf einops pytorch_lightning 

In [ ]:
#@title ### Install xformers from precompiled wheel.
from subprocess import getoutput
from IPython.display import HTML
from IPython.display import clear_output
import time

s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

while True:
    try: 
        gpu=='T4'or gpu=='P100'or gpu=='V100'or gpu=='A100'
        break
    except:
        pass
    print('[1;31mit seems that your GPU is not supported at the moment')
    time.sleep(5)

if (gpu=='T4'):
  %pip install -qq https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
  
elif (gpu=='P100'):
  %pip install -qq https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='V100'):
  %pip install -qq https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='A100'):
  %pip install -qq https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl  

In [ ]:
#@title Download `animefull` model and traindata (nahida)

#@markdown Path of model
URL = "https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-pruned.tar" #@param {type:"string"}

#@markdown Path of the initial model folder.
SRC_PATH = "/content/animefull-pruned" #@param {type:"string"}

#@markdown Path of the convert output (as train model input).
MODEL_NAME = "/content/animefull" #@param {type:"string"}

#@markdown Download example training data (nahida)?
download_example_traindata = True #@param {type:"boolean"}

#@markdown Use external `animevae.pt`?
include_animevae = True #@param {type:"boolean"}

!mkdir -p $SRC_PATH
%cd $SRC_PATH
!apt install -y -qq aria2 > /dev/null
!aria2c --summary-interval=5 -x 6 --allow-overwrite=true -o data.tar $URL
!echo "Decompressing.."
!tar xf data.tar 
!echo "Done"

%cd /content
!curl -Lo convert.py https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/convert_v2.py
!curl -Lo /content/animevae.pt https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animevae.pt

if include_animevae:
  !python convert.py --checkpoint_path $SRC_PATH/model.ckpt --original_config_file $SRC_PATH/config.yaml --vae_path animevae.pt --dump_path $MODEL_NAME --scheduler_type ddim
else:
  !python convert.py --checkpoint_path $SRC_PATH/model.ckpt --original_config_file $SRC_PATH/config.yaml --dump_path $MODEL_NAME --scheduler_type ddim

if download_example_traindata:
  !mkdir -p nahida
  !curl -L https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/train-nahida.tar | tar x -C nahida > /dev/null
  !rm -fr nahida/._train*


In [ ]:
#@title Settings

#@markdown Path and prompt for images of the concept for training.
INSTANCE_PROMPT = "cute nahida" #@param {type:"string"}
INSTANCE_DIR = "/content/nahida" #@param {type:"string"}
!mkdir -p $INSTANCE_DIR

#@markdown A general name and prompt for class like dog for dog images.
CLASS_PROMPT = "full body illustration of a anime cute face young girl in dramatic light, masterpiece and best quality." #@param {type:"string"}
CLASS_NEGATIVE_PROMPT = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry" #@param {type:"string"}
CLASS_DIR = "/content/nahida_autogen" #@param {type:"string"}

#@markdown Prompt for saving samples.
SAVE_SAMPLE_PROMPT = "nahida sitting on the chair, masterpiece and best quality." #@param {type:"string"}
SAVE_SAMPLE_NEGATIVE_PROMPT = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry" #@param {type:"string"}

#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = False #@param {type:"boolean"}

#@markdown Enter the directory name to save model at.
OUTPUT_DIR = "nahida-dreambooth-output" #@param {type:"string"}

if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

In [ ]:
#@title Metrics

#@markdown Use external services to monitor training.
use_tensorboard = False #@param {type:"boolean"}
use_wandb = False #@param {type:"boolean"}
save_weights_to_wandb = False #@param {type:"boolean"}
wandb_apikey = "" #@param {type:"string"}

if use_wandb:
  %pip install -qq git+https://github.com/wandb/wandb
  if wandb_apikey == "":
    raise ValueError('Invalid wandb.ai APIKey')
  !wandb login $wandb_apikey

if use_tensorboard:
  # Tensorboard support
  %load_ext tensorboard
  %tensorboard --logdir $OUTPUT_DIR/logs

### Start Training

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

In [ ]:
#@markdown Use the keywords included in each instance images as unique instance prompt.
read_prompt_from_filename = False #@param {type:"boolean"}
read_prompt_from_txt = False #@param {type:"boolean"}

#@markdown Training parameters.
resolution = 448 #@param {type:"slider", min:64, max:2048, step:32}
train_batch_size = 1 #@param {type:"slider", min:1, max:10, step:1}
seed = 1337 #@param {type:"number"}
num_class_images = 150 #@param {type:"number"}
max_train_steps = 2000 #@param {type:"number"}
save_interval = 500 #@param {type:"number"}
log_interval = 10 #@param {type:"number"}
lr_scheduler = "cosine_with_restarts"  #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"] {allow-input: false}
learning_rate = 5e-6 #@param {type:"number"}


wandb_arg = ""
save_sample_arg = ""
extra_prompt_arg = ""

if read_prompt_from_filename:
  extra_prompt_arg = "--read_prompt_filename"

if read_prompt_from_txt:
  extra_prompt_arg = "--read_prompt_txt"

if use_wandb:
  wandb_arg = "--wandb"

if save_weights_to_wandb:
  wandb_arg = "--wandb --wandb_artifact"

%cd /content
!mkdir -p $OUTPUT_DIR
!accelerate launch train_dreambooth.py \
  --mixed_precision="fp16" \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="{INSTANCE_PROMPT}" \
  --class_prompt="{CLASS_PROMPT}" \
  --class_negative_prompt="{CLASS_NEGATIVE_PROMPT}" \
  --save_sample_prompt={SAVE_SAMPLE_PROMPT} \
  --save_sample_negative_prompt={SAVE_SAMPLE_NEGATIVE_PROMPT} \
  --seed=$seed \
  --resolution=$resolution \
  --train_batch_size=$train_batch_size \
  --gradient_accumulation_steps=1 \
  --learning_rate=$learning_rate \
  --lr_scheduler=$lr_scheduler \
  --lr_warmup_steps=100 \
  --num_class_images=$num_class_images \
  --sample_batch_size=4 \
  --max_train_steps=$max_train_steps \
  --save_interval=$save_interval \
  --log_interval=$log_interval \
  --gradient_checkpointing \
  --use_8bit_adam $wandb_arg $save_sample_arg $extra_prompt_arg

# disabled: --not_cache_latents

In [ ]:
#@title Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

!curl -Lo back_convert.py https://github.com/huggingface/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
!curl -Lo back_convert_alt.py https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/back_convert.py

#@markdown Run conversion.
ckpt_path = OUTPUT_DIR + "/model.ckpt"
src_ckpt_path = SRC_PATH + "/model.ckpt"
use_checkpoint = 'checkpoint_last' #@param {type:"string"}
use_alt_method = False  #@param {type:"boolean"}
half_precision = False  #@param {type:"boolean"}

if use_alt_method:
  !python back_convert_alt.py --model_path '{OUTPUT_DIR}/checkpoint_last' --src_path $src_ckpt_path  --checkpoint_path $ckpt_path 
else:
  if half_precision:
    !python back_convert.py --model_path '{OUTPUT_DIR}/checkpoint_last' --checkpoint_path $ckpt_path --half
  else:
    !python back_convert.py --model_path '{OUTPUT_DIR}/checkpoint_last' --checkpoint_path $ckpt_path 


print(f"[*] Converted ckpt saved at {ckpt_path}")

In [ ]:
#@title Inference

import torch
import os
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

#@markdown Run Gradio UI for generating images.
use_checkpoint = 'checkpoint_last' #@param {type:"string"}
model_path = os.path.join(OUTPUT_DIR, use_checkpoint)

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
g_cuda = None


import gradio as gr

def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of sks guy, digital painting")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=50)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)